In [1]:
!nnunet_prepare_data_folder -h

2024-02-01 12:03:14 - Hive - INFO - Hive GPU Available: True
usage: nnunet_prepare_data_folder [-h] -i INPUT_DATA_FOLDER
                                  [--task-ID TASK_ID] --task-name TASK_NAME
                                  [--test-split [0-100]] --config-file
                                  CONFIG_FILE [-v | -q]

Prepare Dataset folder according to the nnUNet specifications, creating and populating the subfolders ``imagesTr``,
``labelsTr``, ``imagesTs`` and ``labelsTs``. In addition, a JSON instance configuration file (as required by nnUNet)
for each label mask is generated, alongside a summary of the train/test split of the dataset.

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT_DATA_FOLDER, --input-data-folder INPUT_DATA_FOLDER
                        Input Dataset folder
  --task-ID TASK_ID     Task ID used in the folder path tree creation (Default: 100)
  --task-name TASK_NAME
                        Task Name used in the folder pa

In [2]:
!nnunet_run_preprocessing -h

2024-02-01 12:03:21 - Hive - INFO - Hive GPU Available: True
usage: nnunet_run_preprocessing [-h] --config-file CONFIG_FILE [-v | -q]

Run nnUNet command to extract dataset fingerprints, create experiment plan, preprocessing and ( optionally ) verify 
the dataset integrity.
The CL script called is  ``nnUNetv2_plan_and_preprocess``, with the arguments extracted from the given configuration file.

optional arguments:
  -h, --help            show this help message and exit
  --config-file CONFIG_FILE
                        File path for the configuration dictionary, used to retrieve experiments variables (Task_ID) 
  -v, --verbose         Enable verbose output in terminal. Add multiple times to increase verbosity.
  -q, --silent          Suppress most log outputs in terminal.

Example call:
::
    nnunet_run_preprocessing.py --config-file /PATH/TO/CONFIG_FILE.json


In [7]:
%%bash

#python -m monai.bundle init_bundle nnUNet

mkdir nnUNet/bundle_scripts
touch nnUNet/bundle_scripts/__init__.py
which tree && tree nnUNet || true

In [10]:
import monai

monai.config.print_config()

MONAI version: 1.3.0
Numpy version: 1.22.2
Pytorch version: 2.1.0a0+fe05266
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/.local/lib/python3.8/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.0
scikit-image version: 0.21.0
scipy version: 1.10.1
Pillow version: 9.2.0
Tensorboard version: 2.9.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.0a0
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.4
pandas version: 1.5.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.9.2
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please

In [19]:
!pip show nnunetv2

Name: nnunetv2
Version: 2.1.1
Summary: nnU-Net. Framework for out-of-the box biomedical image segmentation.
Home-page: https://github.com/MIC-DKFZ/nnUNet
Author: Helmholtz Imaging Applied Computer Vision Lab, Division of Medical Image Computing, German Cancer Research Center
Author-email: f.isensee@dkfz-heidelberg.de
License: Apache License Version 2.0, January 2004
Location: /opt/code/nnunet
Requires: torch, acvl-utils, dynamic-network-architectures, tqdm, dicom2nifti, scikit-image, scipy, batchgenerators, numpy, scikit-learn, scikit-image, SimpleITK, pandas, graphviz, tifffile, requests, nibabel, matplotlib, seaborn, imagecodecs, yacs, mlflow
Required-by: 


In [24]:
%%writefile nnUNet/configs/metadata.json

{
    "version": "0.1.0",
    "monai_version": "1.3.0",
    "pytorch_version": "2.1.0",
    "numpy_version": "1.22.2",
    "optional_packages_version": {
        "nnunet": "2.1.1"
        
        },
    "task": "nnUNet Bundle",
    "description": "A nnUNet MONAI Bundle, used to run nnUNet Experiments within the MONAI Bundle framework.",
    "authors": "Simone Bendazzoli",
    "data_type": "nifti",
    "references": [
        "Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211."
    ]
    }

Overwriting nnUNet/configs/metadata.json


In [35]:
%%writefile nnUNet/configs/prepare.yaml

imports:
- $import subprocess

data_dir: .
task_id: ""
hive_config_file: ""
task_name: ""

cmd:
- "nnunet_prepare_data_folder"
- "-i"
- '@data_dir'
- "--task-ID"
- '$str(@task_id)'
- "--task-name"
- '@task_name'
- "--test-split"
- "0"
- "--config-file"
- '@hive_config_file'

prepare:
- '$print(@cmd)'
- '$subprocess.run(@cmd)'

Writing nnUNet/configs/prepare.yaml


In [34]:
%%writefile nnUNet/configs/preprocess.yaml

imports:
- $import subprocess
- $from pathlib import Path
- $import os
- $import json

hive_config_file: ""
config_dict: "$json.load(open(@hive_config_file))"
task_id: ""

cmd:
- "nnunet_run_preprocessing"
- "--config-file"
- "$str(Path(os.environ['ROOT_FOLDER']).joinpath(@config_dict['Experiment Name'], @config_dict['Experiment Name'] + '_results','Dataset' + str(@task_id) + '_' + @config_dict['Experiment Name'] + '.json'))"

preprocess:
- '$print(@cmd)'
- '$subprocess.run(@cmd)'

Writing nnUNet/configs/preprocess.yaml


In [51]:
%%writefile nnUNet/bundle_scripts/trainer.py

import torch
import json
from typing import Union, Optional
from torch.backends import cudnn
from pathlib import Path
import os


def get_nnunet_trainer(dataset_name_or_id: Union[str, int],
             configuration: str, fold: Union[int, str],
             hive_config_file: str,                        #To set env variables
             trainer_class_name: str = 'nnUNetTrainer',
             plans_identifier: str = 'nnUNetPlans',
             pretrained_weights: Optional[str] = None,
             num_gpus: int = 1,
             use_compressed_data: bool = False,
             export_validation_probabilities: bool = False,
             continue_training: bool = False,
             only_run_validation: bool = False,
             disable_checkpointing: bool = False,
             val_with_best: bool = False,
             device: torch.device = torch.device('cuda')): # From nnUNet/nnunetv2/run/run_training.py#run_training
    
    ## Block Added
    with open(hive_config_file,"r") as f:
        hive_config_dict = json.load(f)
    
    os.environ["nnUNet_raw"] = str(Path(hive_config_dict["base_folder"]).joinpath("nnUNet_raw_data"))
    os.environ["nnUNet_preprocessed"] = hive_config_dict["preprocessing_folder"]
    os.environ["nnUNet_results"] = hive_config_dict["results_folder"]

    from nnunetv2.run.run_training import get_trainer_from_args, maybe_load_checkpoint
    ##
    
    if isinstance(fold, str):
        if fold != 'all':
            try:
                fold = int(fold)
            except ValueError as e:
                print(f'Unable to convert given value for fold to int: {fold}. fold must bei either "all" or an integer!')
                raise e

   

    if int(num_gpus) > 1:
        ... # Disable for now
    else:
        nnunet_trainer = get_trainer_from_args(str(dataset_name_or_id), configuration, fold, trainer_class_name,
                                               plans_identifier, use_compressed_data, device=device)

        if disable_checkpointing:
            nnunet_trainer.disable_checkpointing = disable_checkpointing

        assert not (continue_training and only_run_validation), f'Cannot set --c and --val flag at the same time. Dummy.'

        maybe_load_checkpoint(nnunet_trainer, continue_training, only_run_validation, pretrained_weights)
        nnunet_trainer.on_train_start() # To Initialize Trainer
        if torch.cuda.is_available():
            cudnn.deterministic = False
            cudnn.benchmark = True

        # Skip Training and Validation Phase

        return nnunet_trainer
    
def prepare_nnunet_batch(batch,device,non_blocking):
    data = batch["data"].to(device, non_blocking=non_blocking)
    if isinstance(batch["target"], list):
        target = [i.to(device, non_blocking=non_blocking) for i in batch["target"]]
    else:
        target = batch["target"].to(device, non_blocking=non_blocking)
    return data, target

Overwriting nnUNet/bundle_scripts/trainer.py


In [69]:
%%writefile nnUNet/configs/train.yaml

imports:
- $from bundle_scripts.trainer import get_nnunet_trainer
- $from bundle_scripts.trainer import prepare_nnunet_batch
- $import json
- $from pathlib import Path
- $import os
- $from ignite.contrib.handlers.tqdm_logger import ProgressBar
- $import shutil

task_id: ""
bundle_root: ""

hive_config_file: ""
hive_config_dict: "$json.load(open(@hive_config_file))"
hive_config_file_final: "$str(Path(os.environ['ROOT_FOLDER']).joinpath(@hive_config_dict['Experiment Name'], @hive_config_dict['Experiment Name'] + '_results','Dataset' + str(@task_id) + '_' + @hive_config_dict['Experiment Name'] + '.json'))"
nnunet_model_folder: "$str(Path(os.environ['ROOT_FOLDER']).joinpath(@hive_config_dict['Experiment Name'], @hive_config_dict['Experiment Name'] + '_results','Dataset' + str(@task_id) + '_' + @hive_config_dict['Experiment Name'],'nnUNetTrainerHive__nnUNetPlans__3d_fullres'))"

nnunet_config:
    device: "$torch.device('cuda')"
    dataset_name_or_id: "@task_id"
    configuration: "3d_fullres"
    fold: 0
    trainer_class_name: "nnUNetTrainerHive"
    num_gpus: "1"
    hive_config_file: "@hive_config_file_final"


nnunet_trainer: "$get_nnunet_trainer(**@nnunet_config)"


# Needed at Inference Time. See inference.yaml
checkpoint:
    init_args: '$@nnunet_trainer.my_init_kwargs'
    trainer_name: '$@nnunet_trainer.__class__.__name__'
    inference_allowed_mirroring_axes: '$@nnunet_trainer.inference_allowed_mirroring_axes'

checkpoint_filename: "$@bundle_root+'/models/nnunet_checkpoint.pth'"

postprocessing:
  _target_: "Compose"
  transforms:
  - _target_: Lambdad
    keys:
    - "pred"
    - "label"
    func: "$lambda x: x[0]"
  - _target_: Activationsd
    keys:
    - "pred"
    softmax: True

    
validation:
    dataloader: "$@nnunet_trainer.dataloader_val"
    pbar:
      _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
    key_metric:
        Val_Dice:
            _target_: "MeanDice"
            output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
            reduction: "mean"
    additional_metrics:
        Val_Dice_per_class:
            _target_: "MeanDice"
            output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
            reduction: "mean_batch"
      
train:
    dataloader: "$@nnunet_trainer.dataloader_train"
    pbar:
      _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
    key_metric:
        Train_Dice:
            _target_: "MeanDice"
            output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
            reduction: "mean_batch"
    handlers:
    - _target_: "LrScheduleHandler"
      lr_scheduler: "$@nnunet_trainer.lr_scheduler"
    - _target_: "ValidationHandler"
      interval: 1
      validator: "$@validator"
      exec_at_start: True
    - _target_: "CheckpointSaver"
      save_dir: "$@bundle_root+'/models'"
      save_interval: 1
      n_saved: 1
      save_dict:
        network_weights: '$@nnunet_trainer.network'
        optimizer_state: '$@nnunet_trainer.optimizer'
        scheduler: '$@nnunet_trainer.lr_scheduler'

validator:
    _target_: "SupervisedEvaluator"
    postprocessing: "$@postprocessing"
    device: "$@nnunet_trainer.device"
    val_data_loader: "$@validation#dataloader"
    network: "$@nnunet_trainer.network"
    key_val_metric: "@validation#key_metric"
    additional_metrics: "$@validation#additional_metrics"
    epoch_length: "$int(@nnunet_trainer.num_val_iterations_per_epoch)"
    prepare_batch: "$prepare_nnunet_batch"

trainer:
    _target_: "SupervisedTrainer"
    postprocessing: "$@postprocessing"
    device: "$@nnunet_trainer.device"
    max_epochs: "$int(@nnunet_trainer.num_epochs)"
    train_data_loader: "$@train#dataloader"
    network: "$@nnunet_trainer.network"
    key_train_metric: "@train#key_metric"    
    optimizer: "$@nnunet_trainer.optimizer"
    loss_function: "$@nnunet_trainer.loss"
    epoch_length: "$int(@nnunet_trainer.num_iterations_per_epoch)"
    prepare_batch: "$prepare_nnunet_batch"
    train_handlers: "$@train#handlers"
    
run:
- "$@train#pbar.attach(@trainer,output_transform=lambda x: {'loss': x[0]['loss']})"
#- "$@validation#pbar.attach(@validator,metric_names=['Val_Dice'])"
- "$torch.save(@checkpoint,@checkpoint_filename)"
- "$shutil.copy(Path(@nnunet_model_folder).joinpath('dataset.json'), @bundle_root+'/models/dataset.json')"
- "$shutil.copy(Path(@nnunet_model_folder).joinpath('plans.json'), @bundle_root+'/models/plans.json')"
- "$@trainer.run()"

Overwriting nnUNet/configs/train.yaml


In [61]:
%%bash

BUNDLE="./nnUNet"

export PYTHONPATH=$BUNDLE
export ROOT_FOLDER="/home/jovyan/Experiments"

python -m monai.bundle run prepare \
    --bundle_root "$BUNDLE" \
    --data_dir "/mnt/cifs/COVID-19_dataset" \
    --task_id "601" \
    --hive_config_file "/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json" \
    --task_name "LungLobeSeg_Demo_nnUNet_3D_fullres" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/prepare.yaml" 

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.8/runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/__init__.py", line 58, in <module>
    load_submodules(sys.modules[__name__], False, exclude_pattern=excludes)
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/utils/module.py", line 211, in load_submodules
    mod = import_module(name)
  File "/usr/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/apps/__init__.py", line 14, in <module>
    from .datasets import CrossValidation, DecathlonDataset, MedNISTDataset, TciaDataset
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/apps/datasets.py", lin

Error while terminating subprocess (pid=2833): 


In [38]:
%%bash

BUNDLE="./nnUNet"

export PYTHONPATH=$BUNDLE
export N_THREADS=1
export ROOT_FOLDER="/home/jovyan/Experiments"

python -m monai.bundle run preprocess \
    --bundle_root "$BUNDLE" \
    --data_dir "/mnt/cifs/COVID-19_dataset" \
    --task_id "601" \
    --hive_config_file "/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json" \
    --task_name "LungLobeSeg_Demo_nnUNet_3D_fullres" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --config_file "$BUNDLE/configs/preprocess.yaml" 

2024-02-01 12:41:30,054 - INFO - --- input summary of monai.bundle.scripts.run ---
2024-02-01 12:41:30,054 - INFO - > config_file: './nnUNet/configs/preprocess.yaml'
2024-02-01 12:41:30,054 - INFO - > meta_file: './nnUNet/configs/metadata.json'
2024-02-01 12:41:30,054 - INFO - > run_id: 'preprocess'
2024-02-01 12:41:30,054 - INFO - > bundle_root: './nnUNet'
2024-02-01 12:41:30,054 - INFO - > data_dir: '/mnt/cifs/COVID-19_dataset'
2024-02-01 12:41:30,055 - INFO - > task_id: 601
2024-02-01 12:41:30,055 - INFO - > hive_config_file: '/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json'
2024-02-01 12:41:30,055 - INFO - > task_name: 'LungLobeSeg_Demo_nnUNet_3D_fullres'
2024-02-01 12:41:30,055 - INFO - ---


2024-02-01 12:41:30,055 - WARNING - Default logging file in nnUNet/configs/logging.conf does not exist, skipping logging.


2024-02-01 12:41:30 - Hive - INFO - Hive GPU Available: True


Fingerprint extraction...
Dataset601_LungLobeSeg_Demo_nnUNet_3D_fullres
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Experiment planning...
Attempting to find 3d_lowres config. 
Current spacing: [1.03       0.71918958 0.71918958]. 
Current patch size: [ 96 160 160]. 
Current median shape: [291.26213592 497.08737864 497.08737864]
Attempting to find 3d_lowres config. 
Current spacing: [1.0609     0.74076526 0.74076526]. 
Current patch size: [ 96 160 160]. 
Current median shape: [282.77877274 482.60910548 482.60910548]
Attempting to find 3d_lowres config. 
Current spacing: [1.092727   0.76298822 0.76298822]. 
Current patch size: [ 96 160 160]. 
Current median shape: [274.54249781 468.55252959 468.55252959]
Attempting to find 3d_lowres config. 
Current spacing: [1.12550881 0.78587787 0.78587787]. 
Current patch size: [ 96 160 160]. 
Current median shape: [266.54611437 454.90536853 454.90536853]
Attempting to find 3d_lowres config. 
Current spacing: 

100%|██████████| 10/10 [03:12<00:00, 19.25s/it]
/home/jovyan/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
  warn_deprecated(argname, msg, warning_category)


['nnunet_run_preprocessing', '--config-file', '/home/jovyan/Experiments/LungLobeSeg_Demo_nnUNet_3D_fullres/LungLobeSeg_Demo_nnUNet_3D_fullres_results/Dataset601_LungLobeSeg_Demo_nnUNet_3D_fullres.json']


In [67]:
%%bash

BUNDLE="./nnUNet"

export PYTHONPATH=$BUNDLE
export ROOT_FOLDER="/home/jovyan/Experiments"

python -m monai.bundle run\
    --bundle_root "$BUNDLE" \
    --task_id "601" \
    --hive_config_file "/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --logging_file "$BUNDLE/configs/logging.conf" \
    --config_file "$BUNDLE/configs/train.yaml" 

2024-02-01 13:36:30,821 - INFO - --- input summary of monai.bundle.scripts.run ---
2024-02-01 13:36:30,821 - INFO - > config_file: './nnUNet/configs/train.yaml'
2024-02-01 13:36:30,821 - INFO - > meta_file: './nnUNet/configs/metadata.json'
2024-02-01 13:36:30,821 - INFO - > logging_file: './nnUNet/configs/logging.conf'
2024-02-01 13:36:30,821 - INFO - > bundle_root: './nnUNet'
2024-02-01 13:36:30,821 - INFO - > task_id: 601
2024-02-01 13:36:30,821 - INFO - > hive_config_file: '/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json'
2024-02-01 13:36:30,821 - INFO - ---


2024-02-01 13:36:30,822 - INFO - Setting logging properties based on config: ./nnUNet/configs/logging.conf.


/home/jovyan/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
  warn_deprecated(argname, msg, warning_category)


Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 2, 'patch_size': [32, 32, 32], 'median_image_size_in_voxels': [300.0, 512.0, 512.0], 'spacing': [1.0, 0.6982423067092896, 0.6982423067092896], 'normalization_schemes': ['CTNormalization'], 'use_mask_for_norm': [False], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_pe

Epoch [1/1000]: [249/250] 100%|█████████▉, loss=1.02 [00:13<00:00]/home/jovyan/.local/lib/python3.8/site-packages/monai/engines/workflow.py:250: UserWarning: Key metric is not a scalar value, skip the metric comparison with the current best metric.Please set other metrics as the key metric, or change the `reduction` mode to 'mean'.
  warnings.warn(


2024-02-01 13:36:54,273 - ignite.engine.engine.SupervisedTrainer - INFO - Epoch[1] Complete. Time taken: 00:00:18.064
2024-02-01 13:37:07,915 - ignite.engine.engine.SupervisedTrainer - INFO - Current learning rate: 0.009981998198678612
2024-02-01 13:37:07,915 - ignite.engine.engine.SupervisedEvaluator - INFO - Engine run resuming from iteration 0, epoch 1 until 2 epochs
2024-02-01 13:37:09,247 - ignite.engine.engine.SupervisedEvaluator - INFO - Epoch[2] Complete. Time taken: 00:00:01.332
2024-02-01 13:37:09,247 - ignite.engine.engine.SupervisedEvaluator - INFO - Engine run complete. Time taken: 00:00:01.332
2024-02-01 13:37:09,523 - ignite.engine.engine.SupervisedTrainer - INFO - Saved checkpoint at epoch: 2
2024-02-01 13:37:09,524 - ignite.engine.engine.SupervisedTrainer - INFO - Epoch[2] Complete. Time taken: 00:00:15.251
2024-02-01 13:37:23,262 - ignite.engine.engine.SupervisedTrainer - INFO - Current learning rate: 0.009972995945537971
2024-02-01 13:37:23,262 - ignite.engine.engine

In [8]:
%%writefile nnUNetTest/bundle_scripts/inference.py
import torch
import json
import os
from pathlib import Path
from batchgenerators.utilities.file_and_folder_operations import join


from batchgenerators.utilities.file_and_folder_operations import load_json, join, isfile, maybe_mkdir_p, isdir, subdirs, save_json

def initialize_from_trained_model_folder( predictor,
                                          model_training_output_dir: str,
                                             use_folds,
                                             nnunet_checkpoint_name: str = 'nnunet_checkpoint.pth', monai_checkpoint_name: str = 'model.pt'):
    
        from nnunetv2.utilities.plans_handling.plans_handler import PlansManager, ConfigurationManager
        from nnunetv2.utilities.label_handling.label_handling import determine_num_input_channels
        from nnunetv2.utilities.find_class_by_name import recursive_find_python_class
        import nnunetv2
        if use_folds is None:
            use_folds = nnUNetPredictor.auto_detect_available_folds(model_training_output_dir, checkpoint_name)

        dataset_json = load_json(join(model_training_output_dir, 'dataset.json'))
        plans = load_json(join(model_training_output_dir, 'plans.json'))
        plans_manager = PlansManager(plans)

        if isinstance(use_folds, str):
            use_folds = [use_folds]

        parameters = []
        for i, f in enumerate(use_folds):
            f = int(f) if f != 'all' else f
            checkpoint = torch.load(join(model_training_output_dir, nnunet_checkpoint_name),
                                    map_location=torch.device('cpu'))
            if i == 0:
                trainer_name = checkpoint['trainer_name']
                configuration_name = checkpoint['init_args']['configuration']
                inference_allowed_mirroring_axes = checkpoint['inference_allowed_mirroring_axes'] if \
                    'inference_allowed_mirroring_axes' in checkpoint.keys() else None

            
            monai_checkpoint = torch.load(join(model_training_output_dir, monai_checkpoint_name),
                                    map_location=torch.device('cpu'))
            parameters.append(monai_checkpoint['network_weights'])
            
        configuration_manager = plans_manager.get_configuration(configuration_name)
        # restore network
        num_input_channels = determine_num_input_channels(plans_manager, configuration_manager, dataset_json)
        trainer_class = recursive_find_python_class(join(nnunetv2.__path__[0], "training", "nnUNetTrainer"),
                                                    trainer_name, 'nnunetv2.training.nnUNetTrainer')
        network = trainer_class.build_network_architecture(plans_manager, dataset_json, configuration_manager,
                                                           num_input_channels, enable_deep_supervision=False)
        predictor.plans_manager = plans_manager
        predictor.configuration_manager = configuration_manager
        predictor.list_of_parameters = parameters
        predictor.network = network
        predictor.dataset_json = dataset_json
        predictor.trainer_name = trainer_name
        predictor.allowed_mirroring_axes = inference_allowed_mirroring_axes
        predictor.label_manager = plans_manager.get_label_manager(dataset_json)
        if ('nnUNet_compile' in os.environ.keys()) and (os.environ['nnUNet_compile'].lower() in ('true', '1', 't')) \
                and not isinstance(self.network, OptimizedModule):
            print('Using torch.compile')
            predictor.network = torch.compile(predictor.network)


def predict(model_folder, folds, input_folder, output_folder,hive_config_file):
    with open(hive_config_file,"r") as f:
        hive_config_dict = json.load(f)
    
    os.environ["nnUNet_raw"] = str(Path(hive_config_dict["base_folder"]).joinpath("nnUNet_raw_data"))
    os.environ["nnUNet_preprocessed"] = hive_config_dict["preprocessing_folder"]
    os.environ["nnUNet_results"] = hive_config_dict["results_folder"]
    from nnunetv2.paths import nnUNet_results, nnUNet_raw
    from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
# instantiate the nnUNetPredictor
    predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_gpu=True,
        device=torch.device('cuda', 0),
        verbose=False,
        verbose_preprocessing=False,
        allow_tqdm=True
    )
    
    initialize_from_trained_model_folder(predictor,model_folder,folds,)
    
    predictor.predict_from_files(input_folder,
                                 output_folder,    
                             save_probabilities=False, overwrite=False,
                                 num_processes_preprocessing=2, num_processes_segmentation_export=2,
                                 folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)



Overwriting nnUNetTest/bundle_scripts/inference.py


In [6]:
%%writefile nnUNetTest/configs/inference.yaml

imports:
- $from bundle_scripts.train import get_nnunet_trainer
- $from bundle_scripts.train import prepare_nnunet_batch
- $import json
- $from pathlib import Path
- $import os
- $from bundle_scripts.inference import predict

task_id: 100

hive_config_file: ""
hive_config_dict: "$json.load(open(@hive_config_file))"
hive_config_file_final: "$str(Path(os.environ['ROOT_FOLDER']).joinpath(@hive_config_dict['Experiment Name'], @hive_config_dict['Experiment Name'] + '_results','Dataset' + str(@task_id) + '_' + @hive_config_dict['Experiment Name'] + '.json'))"

args:
  input_folder: "@input_folder"
  model_folder: "@model_folder"
  output_folder: "@output_folder"
  folds: "0"
  hive_config_file: "@hive_config_file_final"

inference:
- '$predict(**@args)'

Writing nnUNetTest/configs/inference.yaml


In [7]:
%%bash

BUNDLE="./nnUNetTest"

export PYTHONPATH=$BUNDLE
export ROOT_FOLDER=$BUNDLE"/Experiments"

python -m monai.bundle run inference\
    --bundle_root "$BUNDLE" \
    --task_id "601" \
    --hive_config_file "/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json" \
    --model_folder "/home/jovyan/LungLobeSeg_nnUNet_3D_fullres/results/Dataset102_LungLobeSeg_nnUNet_3D_fullres/nnUNetTrainerHive__nnUNetPlans__3d_fullres"\
    --input_folder "/home/jovyan/MONAI/nnUNetTest/Experiments/LungLobeSeg_Demo_nnUNet_3D_fullres/LungLobeSeg_Demo_nnUNet_3D_fullres_base/nnUNet_raw_data/Dataset601_LungLobeSeg_Demo_nnUNet_3D_fullres/imagesTr"\
    --output_folder "/home/jovyan/output" \
    --meta_file "$BUNDLE/configs/metadata.json" \
    --logging_file "$BUNDLE/configs/logging.conf" \
    --config_file "$BUNDLE/configs/inference.yaml" 

2024-01-06 12:45:59,955 - INFO - --- input summary of monai.bundle.scripts.run ---
2024-01-06 12:45:59,955 - INFO - > config_file: './nnUNetTest/configs/inference.yaml'
2024-01-06 12:45:59,956 - INFO - > meta_file: './nnUNetTest/configs/metadata.json'
2024-01-06 12:45:59,956 - INFO - > logging_file: './nnUNetTest/configs/logging.conf'
2024-01-06 12:45:59,956 - INFO - > run_id: 'inference'
2024-01-06 12:45:59,956 - INFO - > bundle_root: './nnUNetTest'
2024-01-06 12:45:59,956 - INFO - > task_id: 601
2024-01-06 12:45:59,956 - INFO - > hive_config_file: '/home/jovyan/LungLobeSeg_Demo_nnUNet_3D_fullres.json'
2024-01-06 12:45:59,956 - INFO - > model_folder: '/home/jovyan/LungLobeSeg_nnUNet_3D_fullres/results/Dataset102_LungLobeSeg_nnUNet_3D_fullres/nnUNetTrainerHive__nnUNetPlans__3d_fullres'
2024-01-06 12:45:59,956 - INFO - > input_folder: '/home/jovyan/MONAI/nnUNetTest/Experiments/LungLobeSeg_Demo_nnUNet_3D_fullres/LungLobeSeg_Demo_nnUNet_3D_fullres_base/nnUNet_raw_data/Dataset601_LungLobeS

/home/jovyan/.local/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.bundle.workflows ConfigWorkflow.__init__:workflow_type: Current default value of argument `workflow_type=None` has been deprecated since version 1.2. It will be changed to `workflow_type=train` in version 1.4.
  warn_deprecated(argname, msg, warning_category)


There are 10 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 10 cases that I would like to predict
overwrite was set to False, so I am only working on cases that haven't been predicted yet. That's 10 cases.
Error while terminating subprocess (pid=148): 


Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/bundle/__main__.py", line 31, in <module>
    fire.Fire()
  File "/home/jovyan/.local/lib/python3.8/site-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/home/jovyan/.local/lib/python3.8/site-packages/fire/core.py", line 475, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/home/jovyan/.local/lib/python3.8/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/bundle/scripts.py", line 787, in run
    workflow.run()
  File "/home/jovyan/.local/lib/python3.8/site-packages/monai/bundle